In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import qiskit
from qiskit import QuantumCircuit, Aer, transpile, IBMQ, execute

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-sherbrooke')

In [ ]:
statevector_sim = Aer.get_backend('statevector_simulator')
qasm_sim = provider.get_backend('ibmq_qasm_simulator')
local_qasm_sim = Aer.get_backend('qasm_simulator')
sydney = provider.get_backend('ibmq_sydney')

In [ ]:
PI = np.pi
SEED = 7531
ZERO_STATE = 0
XY_STEPS = 20
Z_STEPS = 40
SHOTS = 8192

unit_rotation = PI/20
xy_rotations = [unit_rotation * i for i in range(XY_STEPS)]
z_rotations = [unit_rotation * i for i in range(Z_STEPS)]

In [ ]:
def create_xrot_circuit(angle):
    qc = QuantumCircuit(1)
    qc.rx(angle, 0)
    return qc

def create_yrot_circuit(angle):
    qc = QuantumCircuit(1)
    qc.ry(angle, 0)
    return qc

def create_zrot_circuit(angle):
    qc = QuantumCircuit(1)
    qc.h(0)
    qc.rz(angle, 0)
    qc.h(0)
    return qc

xrot_circuits = list(map(lambda r: create_xrot_circuit(r), xy_rotations))
yrot_circuits = list(map(lambda r: create_yrot_circuit(r), xy_rotations))
zrot_circuits = list(map(lambda r: create_zrot_circuit(r), z_rotations))

## Compute the expected 0 state

In [ ]:
states = execute(xrot_circuits + yrot_circuits + zrot_circuits, statevector_sim).result()

In [ ]:
zero_ampl = [r.data.statevector[ZERO_STATE] for r in states.results]
zero_prob = [np.abs(a)**2 for a in zero_ampl]
xrot_zero_prob = zero_prob[:len(xy_rotations)]
yrot_zero_prob = zero_prob[len(xy_rotations):-len(z_rotations)]
zrot_zero_prob = zero_prob[-len(z_rotations):]

## Compute with QASM simulator -- no noise

In [ ]:
for c in xrot_circuits + yrot_circuits + zrot_circuits:
    c.measure_all()

In [ ]:
results = execute(xrot_circuits + yrot_circuits + zrot_circuits,
                  local_qasm_sim,
                  optimization_level=0, 
                  seed_simulator=SEED,
                  shots=SHOTS).result()

In [ ]:
results = provider.backends.retrieve_job('60393b196390d14a08859f53').result()

In [ ]:
xrot_qasm_samp = [r['0']/SHOTS for r in results.get_counts()[:len(xy_rotations)]]
yrot_qasm_samp = [r['0']/SHOTS for r in results.get_counts()[len(xy_rotations):-len(z_rotations)]]
zrot_qasm_samp = [r['0']/SHOTS if '0' in r.keys() else 0. for r in results.get_counts()[-len(z_rotations):]]

## Qubit with lowest single-qubit gate error rate

At the time of this experiment, qubit 14

In [ ]:
results = provider.backends.retrieve_job('60393b392f62bf4e3bc6a9e6').result()

In [ ]:
xrot_sydney_best = [r['0']/SHOTS for r in results.get_counts()[:len(xy_rotations)]]
yrot_sydney_best = [r['0']/SHOTS for r in results.get_counts()[len(xy_rotations):-len(z_rotations)]]
zrot_sydney_best = [r['0']/SHOTS if '0' in r.keys() else 0. for r in results.get_counts()[-len(z_rotations):]]

## Qubit with highest single-qubit gate error rate

At the time of this experiment, qubit 14

In [ ]:
results = provider.backends.retrieve_job('60395a94fcca6bbd3c4c57b0').result()

In [ ]:
xrot_sydney_worst = [r['0']/SHOTS for r in results.get_counts()[:len(xy_rotations)]]
yrot_sydney_worst = [r['0']/SHOTS for r in results.get_counts()[len(xy_rotations):-len(z_rotations)]]
zrot_sydney_worst = [r['0']/SHOTS if '0' in r.keys() else 0. for r in results.get_counts()[-len(z_rotations):]]

In [ ]:
xlabels = ['RX angle (rad)','RY angle (rad)','RZ angle (rad)']
ylabels = ['Probability of measuring\n0 state (Z-basis)', 'Probability of measuring\n0 state (Z-basis)', 'Probability of measuring\n0 state (X-basis)']
xdata = [xy_rotations, xy_rotations, z_rotations]
ydata = [[xrot_zero_prob, xrot_qasm_samp, xrot_sydney_best, xrot_sydney_worst],
         [yrot_zero_prob, yrot_qasm_samp, yrot_sydney_best, yrot_sydney_worst],\
         [zrot_zero_prob, zrot_qasm_samp, zrot_sydney_best, zrot_sydney_worst], ]

fig, axs = plt.subplots(3, figsize=(12,12))

for i in range(3):
    axs[i].plot(xdata[i], ydata[i][0], label='statevector simulator', color='midnightblue', marker='o', linestyle='dotted', linewidth=2, markersize=4)
    axs[i].plot(xdata[i], ydata[i][1], label='qasm simulator', color='slateblue', marker='o', linestyle='dotted', linewidth=2, markersize=4)
    axs[i].plot(xdata[i], ydata[i][2], label='sydney noisiest gate', color='purple', marker='o', linestyle='dotted', linewidth=2, markersize=4)
    axs[i].plot(xdata[i], ydata[i][3], label='sydney best quality gate', color='plum', marker='o', linestyle='dotted', linewidth=2, markersize=4)
    axs[i].set_xlim(0, xdata[i][-1])
    axs[i].set_ylim(0, 1.05)
    axs[i].set_xlabel(xlabels[i])
    axs[i].set_ylabel(ylabels[i])
    axs[i].grid(True)
    axs[i].legend()

fig.tight_layout()
plt.savefig('discrete_rotations')